In [1]:
import os

from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
print("OpenAI API 키가 설정되었습니다.")


data_path = "./data"

OpenAI API 키가 설정되었습니다.


# DOCX 파일 전처리 실습
## Microsoft Word 문서(.docx) 데이터 추출 및 정제

이 섹션에서는 **Microsoft Word 문서(.docx)** 파일로부터 데이터를 추출하고 정제하는 과정을 실습합니다.   
DOCX 파일은 비정형 데이터의 대표적인 형태로, 텍스트, 표, 이미지, 서식 등 다양한 요소가 포함되어 있어 체계적인 전처리가 필요합니다.

### 실습 목표
- `python-docx` 라이브러리를 활용한 DOCX 파일 구조 분석
- 문서 내 텍스트, 표, 서식 정보 추출 방법 학습
- 추출된 데이터의 정제 및 구조화 과정 이해
- LangChain Document 객체로 변환하여 RAG 시스템 활용 준비

### 예제 파일 정보
- **파일명**: `Delivery-Plan-Cabinet-report-South.docx`
- **내용**: 영국 지방자치단체에서 제공하는 실제 의회 보고서 문서
- **특징**: 정책 성과 측정 및 관리 방법에 대한 공식 문서로, 다양한 서식과 표가 포함된 실제 업무 문서

### 사용 라이브러리
- **python-docx**: Microsoft Word 문서(.docx) 파일을 읽고 조작하는 Python 라이브러리
- **LangChain**: 추출된 텍스트를 Document 객체로 변환하여 RAG 시스템에서 활용

### DOCX 파일 구조
DOCX 파일은 다음과 같은 요소들로 구성됩니다:
- **문단(Paragraphs)**: 텍스트의 기본 단위, 각각 고유한 스타일 정보 포함
- **표(Tables)**: 행과 열로 구성된 데이터 구조
- **이미지(Images)**: 문서에 삽입된 이미지 파일
- **서식(Styles)**: 제목, 본문, 목록 등 문서의 구조적 정보
- **메타데이터**: 문서 작성자, 생성일, 수정일 등 부가 정보

## DOCX 파일 로딩 및 기본 구조 분석
`python-docx` 라이브러리를 활용하여 DOCX 파일을 로드하고 문서의 기본 구조를 파악합니다.

In [ ]:
from docx import Document

file_name = "Delivery-Plan-Cabinet-report-South.docx"
doc_path = f"{data_path}/{file_name}"
doc = Document(doc_path)

In [ ]:
# 문서 내 문단 출력
print("문서 내 문단 수:", len(doc.paragraphs))
print("\n첫 10개 문단의 텍스트와 스타일:")
for i, para in enumerate(doc.paragraphs[:10]):
    print(f"\n[{i}번째 문단]")
    print(f"스타일: {para.style.name}")
    print(f"텍스트: {para.text[:100]}...")  # 텍스트가 긴 경우 앞부분만 출력

# 문서 내 모든 표 정보 출력
print(f"\n\n문서 내 표 수: {len(doc.tables)}")
if len(doc.tables) > 0:
    print("\n첫 번째 표의 행/열 정보:")
    first_table = doc.tables[0]
    print(f"행 수: {len(first_table.rows)}")
    print(f"열 수: {len(first_table.columns)}")

## 문서 구조 기반 텍스트 추출 및 Markdown 변환
### 서식 정보를 활용한 구조화된 텍스트 추출

DOCX 파일의 가장 큰 특징은 **서식 정보**를 포함하고 있다는 것입니다.   
문서의 구조적 정보(제목, 본문, 목록 등)를 활용하여 Markdown 형태로 변환하는 함수를 만듭니다.  

- **구조 보존**: 원본 문서의 계층적 구조를 Markdown 헤딩으로 표현
- **가독성 향상**: 표, 목록 등 구조적 요소를 Markdown 문법으로 변환
- **RAG 활용**: LangChain Document 객체로 변환하여 검색 및 질의응답에 활용

### 변환 규칙
1. **제목 변환**: Heading 1~4 → # ~ ####
2. **목록 변환**: List 스타일 → * (불릿 포인트)
3. **표 변환**: Table 객체 → Markdown 테이블 문법
4. **본문**: Normal 스타일 → 일반 텍스트

### 핵심 함수들
- `convert_text_to_markdown()`: 문단의 스타일에 따라 Markdown으로 변환
- `convert_table_to_markdown()`: 표를 Markdown 테이블 형태로 변환
- **문서 순서 보존**: 문단과 표를 문서 내 실제 순서대로 정렬하여 추출

In [ ]:
# 텍스트를 마크다운으로 변환


def convert_text_to_markdown(para):
    markdown_text = ""

    # 스타일에 따른 마크다운 변환
    style_name = para.style.name.lower()

    if "heading 1" in style_name:
        markdown_text += f"\n# {para.text}\n"
    elif "heading 2" in style_name:
        markdown_text += f"\n## {para.text}\n"
    elif "heading 3" in style_name:
        markdown_text += f"\n### {para.text}\n"
    elif "heading 4" in style_name:
        markdown_text += f"\n#### {para.text}\n"
    elif "list" in style_name:
        markdown_text += f"* {para.text}\n"
    else:
        markdown_text += f"{para.text}\n"

    return markdown_text

In [ ]:
# 표를 마크다운으로 변환


def convert_table_to_markdown(table):
    markdown_text = ""

    header_row = []
    for cell in table.rows[0].cells:
        header_row.append(cell.text.strip())

    # 헤더 행 마크다운 작성
    markdown_text += "| " + " | ".join(header_row) + " |\n"

    # 구분선 추가
    markdown_text += "|" + "|".join(["---" for _ in header_row]) + "|\n"

    # 데이터 행 추출
    for row in table.rows[1:]:
        row_data = []
        for cell in row.cells:
            row_data.append(cell.text.strip())
        markdown_text += "| " + " | ".join(row_data) + " |\n"

    markdown_text += "\n"  # 테이블 간 구분을 위한 빈 줄

    return markdown_text

In [ ]:
# 문서 내 모든 블록(문단과 표)을 순서대로 추출


all_blocks = []

# 문단 객체들을 리스트에 추가
for paragraph in doc.paragraphs:
    all_blocks.append(("paragraph", paragraph))

# 표 객체들을 리스트에 추가
for table in doc.tables:
    all_blocks.append(("table", table))

# 객체들을 문서 내 순서대로 정렬
all_blocks.sort(key=lambda x: x[1]._element.getparent().index(x[1]._element))

markdown_text = ""

for block_type, block in all_blocks:
    if block_type == "paragraph":
        markdown_text += convert_text_to_markdown(block)
    else:
        markdown_text += convert_table_to_markdown(block)

print(markdown_text[:3000])

### 불필요한 요소 제거

추출된 Markdown 텍스트에 남아있는 불필요하게 반복된 공백 줄을 제거합니다.

In [ ]:
# 여러 번 연속된 공백 줄을 하나로 통합
markdown_text = "\n".join(
    [
        line
        for line in markdown_text.split("\n")
        if line.strip()
        or not all(
            not l.strip()
            for l in markdown_text.split("\n")[
                markdown_text.split("\n").index(line) - 1 : markdown_text.split("\n").index(line) + 1
            ]
        )
    ]
)

print(markdown_text[:3000])

## LangChain Document 객체로 변환
### RAG 시스템 활용을 위한 표준화된 문서 형태 변환

추출하고 정제한 Markdown 텍스트를 **LangChain Document 객체**로 변환합니다.    
이는 RAG(Retrieval-Augmented Generation) 시스템에서 문서를 표준화된 형태로 처리하기 위한 필수 단계입니다.

### Document 객체 구조
- **page_content**: 실제 텍스트 내용 (정제된 Markdown)
- **metadata**: 문서 메타데이터 (제목, 출처, 파일 경로 등)

In [ ]:
from langchain.docstore.document import Document

# 텍스트를 LangChain Document 객체로 변환
documents = Document(page_content=markdown_text, metadata={"title": file_name, "source": doc_path})

## RAG 시스템 구축 및 테스트
### Retrieval-Augmented Generation 시스템 실습

이제 전처리된 DOCX 문서를 활용하여 **RAG 시스템**을 구축하고 실제 질의응답을 테스트합니다.  

### RAG 시스템 구성 요소
1. **LLM (Large Language Model)**: `gpt-4o-mini` - 질의응답 생성
2. **Embedding Model**: - 텍스트를 벡터로 변환
3. **Vector Database**: FAISS - 벡터 저장 및 유사도 검색
4. **Text Splitter**: RecursiveCharacterTextSplitter - 긴 문서를 청크로 분할

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LLM component
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Embedding component
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Vectorstore component
embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# 1. Indexing
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 문자 수
    chunk_overlap=200,  # 청크 간 중복되는 문자 수 (문맥 유지를 위함)
    add_start_index=True,  # 원본 문서에서의 시작 위치를 추적할지 여부
)
splits = text_splitter.split_documents([documents])
document_ids = vector_store.add_documents(documents=splits)


def retreive_and_generate(query):
    # 2. Retrieval
    docs = vector_store.similarity_search(query)
    context_texts = []
    for doc in docs:
        metadata_str = "\n".join(f"{k}: {v}" for k, v in doc.metadata.items())
        context_texts.append(f"Content:\n{doc.page_content}\n\nMetadata:\n{metadata_str}")

    # 3. Generation
    template = """다음 문맥을 바탕으로 사용자의 질문에 자세히 답변해주세요.
    답변은 반드시 한국어로 작성해야 합니다.
    문맥에 관련된 내용이 없다면, '주어진 문맥에서는 해당 질문에 대한 답변을 찾을 수 없습니다.'라고 답변하세요.

    문맥: {context}
    질문: {question}
    응답:"""

    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm

    response = chain.invoke({"context": "\n\n".join(context_texts), "question": query})
    return response.content

In [ ]:
question = "정책 성과를 어떻게 측정하고 관리하나요?"
retreive_and_generate(question)

# PPT 파일 전처리 실습
## Microsoft PowerPoint 문서(.pptx) 데이터 추출 및 구조화

이 섹션에서는 **Microsoft PowerPoint 문서(.pptx)** 파일로부터 텍스트와 이미지를 슬라이드 단위로 추출하고, 이를 Markdown 형태로 구조화하는 과정을 실습합니다.   
PPT 파일은 프레젠테이션의 특성상 시각적 요소와 텍스트가 복합적으로 구성되어 있어 체계적인 전처리가 필요합니다.

### 실습 목표
- `python-pptx` 라이브러리를 활용한 PPT 파일 구조 분석
- 슬라이드별 텍스트 및 이미지 요소 추출 방법 학습
- 프레젠테이션 구조를 반영한 Markdown 변환 과정 이해
- 시각적 요소와 텍스트의 관계를 고려한 데이터 구조화

### 예제 파일 정보
- **파일명**: `[이론3] 다양한 문서 처리 Tool 구현.pptx`
- **내용**: AI Agent 과정의 이론 강의 자료
- **특징**: 정형 데이터 vs 비정형 데이터 비교, RAG 시스템 설명 등 교육용 프레젠테이션

### 사용 라이브러리
- **python-pptx**: Microsoft PowerPoint 문서(.pptx) 파일을 읽고 조작하는 Python 라이브러리
- **PIL (Pillow)**: 이미지 처리 및 저장을 위한 라이브러리

### PPT 파일 구조
PowerPoint 파일은 다음과 같은 요소들로 구성됩니다:
- **슬라이드(Slides)**: 프레젠테이션의 기본 단위
- **텍스트 박스(Text Boxes)**: 슬라이드 내 텍스트 요소
- **이미지(Images)**: 슬라이드에 삽입된 이미지 파일
- **도형(Shapes)**: 그래프, 차트, 도형 등 시각적 요소
- **마스터 슬라이드**: 전체 프레젠테이션의 일관된 디자인 템플릿

## PPT 파일 로딩 및 기본 구조 분석
`python-pptx` 라이브러리를 활용하여 PPT 파일을 로드하고 슬라이드별 구조를 파악합니다.

In [ ]:
from pptx import Presentation

ppt_file = "[이론3] 다양한 문서 처리 Tool 구현.pptx"
file_path = f"{data_path}/{ppt_file}"

prs = Presentation(file_path)

for i, slide in enumerate(prs.slides):
    print(f"-------- slide {i} --------")
    for shape in slide.shapes:
        if hasattr(shape, "text"):
            print(shape.text)
        elif hasattr(shape, "image"):
            print(shape.image)

## 데이터 전처리 및 구조화
### 슬라이드별 텍스트 추출 및 이미지 처리

PPT 파일의 각 슬라이드에서 텍스트와 이미지를 추출하고, 이미지를 별도로 저장합니다.

### 전처리 목표
- **텍스트 추출**: 슬라이드 내 모든 텍스트 요소 수집
- **이미지 분리**: 슬라이드에 포함된 이미지를 별도 파일로 저장
- **메타데이터 구성**: 슬라이드 번호, 이미지 경로(`image_path`) 등 부가 정보 관리
- **구조화**: 슬라이드별로 체계적인 데이터 구조 생성

### 이미지 처리 과정
1. **이미지 추출**: 슬라이드 내 이미지 요소를 바이트 데이터로 추출
2. **파일 저장**: PIL을 활용하여 PNG 형태로 저장
3. **경로 관리**: 슬라이드 번호와 이미지 순서를 반영한 파일명 생성
4. **오류 처리**: 이미지 추출 실패 시 경고 메시지 출력

### 데이터 구조
각 슬라이드는 다음과 같은 정보를 포함합니다:
- **page_number**: 슬라이드 순서 (0부터 시작)
- **text**: 슬라이드 내 모든 텍스트 (줄바꿈으로 구분)
- **image_paths**: 추출된 이미지 파일들의 경로 리스트

In [ ]:
import io
import os

from PIL import Image

# 이미지 저장 경로 설정
image_dir = "./images/documents_tools"
os.makedirs(image_dir, exist_ok=True)

# 슬라이드별 메타데이터를 저장할 리스트
slides = []

for slide_idx, slide in enumerate(prs.slides):
    slide_text = []
    image_paths = []

    # 슬라이드 내 모든 shape 순회
    for shape in slide.shapes:
        # 텍스트 추출
        if hasattr(shape, "text") and shape.text.strip():
            slide_text.append(shape.text)

        # 이미지 추출 및 저장
        if hasattr(shape, "image"):
            try:
                # 이미지 데이터 추출
                image_bytes = shape.image.blob
                image = Image.open(io.BytesIO(image_bytes))

                # 이미지 파일명 생성 (페이지 번호 포함)
                image_filename = f"slide_{slide_idx:03d}_img_{len(image_paths):02d}.png"
                image_path = os.path.join(image_dir, image_filename)

                # 이미지 저장
                image.save(image_path)
                image_paths.append(image_path)
            except:
                print(f"Warning: 슬라이드 {slide_idx}의 이미지 저장 실패")

    # 슬라이드 메타데이터 저장
    slides.append(
        {
            "page_number": slide_idx,
            "text": "\n".join(slide_text),  # markdown 형식으로 텍스트 저장
            "image_paths": image_paths,
        }
    )

In [ ]:
# 결과 확인
print(f"총 {len(slides)}개의 슬라이드 처리 완료")
for meta in slides[:5]:  # 처음 2개 슬라이드 메타데이터 출력
    print(f"\n슬라이드 {meta['page_number']}:")
    print(f"텍스트: {meta['text'][:100]}...")  # 처음 100자만 출력
    print(f"이미지 경로: {meta['image_paths']}")

## LangChain Document 변환

In [ ]:
from langchain.schema import Document

documents = []

for slide in slides:
    metadata = {"page_number": slide["page_number"], "image_paths": slide["image_paths"]}
    document = Document(page_content=slide["text"], metadata=metadata)
    documents.append(document)

## RAG text

In [ ]:
import base64

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from openai import OpenAI

client = OpenAI()

# LLM component
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Embedding component
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Vectorstore component
embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# 1. Indexing
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 문자 수
    chunk_overlap=200,  # 청크 간 중복되는 문자 수 (문맥 유지를 위함)
    add_start_index=True,  # 원본 문서에서의 시작 위치를 추적할지 여부
)
splits = text_splitter.split_documents(documents)
document_ids = vector_store.add_documents(documents=splits)


def retreive_and_generate(query):
    # 2. Retrieval
    docs = vector_store.similarity_search(query)

    # Prompt 메시지 구성
    messages = [{"role": "user", "content": []}]
    text_parts = []

    for doc in docs:
        # 텍스트 컨텐츠 추가
        text_parts.append(f"Content:\n{doc.page_content}")

        # 이미지가 있는 경우 처리
        if "image_paths" in doc.metadata and doc.metadata["image_paths"]:
            for image_path in doc.metadata["image_paths"]:
                try:
                    base64_image = encode_image_to_base64(image_path)
                    messages[0]["content"].append(
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                    )
                except Exception as e:
                    print(f"이미지 로딩 실패: {image_path}, 에러: {str(e)}")

    # 텍스트 컨텐츠를 메시지에 추가
    text_content = "\n\n".join(text_parts)
    messages[0]["content"].append(
        {
            "type": "text",
            "text": f"""다음 문맥과 제공된 이미지들을 바탕으로 질문에 자세히 답변해주세요.
        답변은 반드시 한국어로 작성해야 합니다.
        문맥에 관련된 내용이 없다면, '주어진 문맥에서는 해당 질문에 대한 답변을 찾을 수 없습니다.'라고 답변하세요.
        만약 내가 이미지를 함께 전달했다면, 해당 이미지에 대한 설명을 먼저 해주세요.

        문맥: {text_content}
        질문: {query}""",
        }
    )

    # GPT-4o를 사용한 응답 생성
    response = client.chat.completions.create(model="gpt-4o-mini", messages=messages, max_tokens=1000)

    return response.choices[0].message.content

In [ ]:
question = "비정형 데이터에 대해서 알려주세요."
print(retreive_and_generate(question))

# XLSX 파일 전처리 실습
## Microsoft Excel 문서(.xlsx) 데이터 분석 및 자연어 질의응답

이 섹션에서는 **Microsoft Excel 문서(.xlsx)** 파일을 Pandas로 로드하고, LangChain의 `create_pandas_dataframe_agent`를 활용하여 사용자의 자연어 질문에 대해 엑셀 데이터를 기반으로 응답하는 시스템을 구현합니다.   
정형 데이터의 대표적인 형태인 스프레드시트를 AI Agent로 분석하는 방법을 학습합니다.

### 실습 목표
- Pandas를 활용한 Excel 파일 로딩 및 기본 데이터 분석
- LangChain DataFrame Agent의 개념과 활용 방법 이해
- 자연어로 데이터베이스 질의를 수행하는 시스템 구축
- 정형 데이터의 AI 기반 분석 및 시각화 방법 학습

### 예제 파일 정보
- **파일명**: `2024학년도+2학기+온라인+수업+중간+강의평가+결과.xlsx`
- **내용**: 학생들의 강의평가 결과 데이터
- **특징**: 강의평점, 학생 정보, 과목 정보 등 교육 데이터로 구성된 실제 데이터셋

### 사용 라이브러리
- **pandas**: Excel 파일 읽기 및 데이터 조작
- **LangChain**: DataFrame Agent 생성 및 자연어 질의처리
- **OpenAI GPT-4o-mini**: 자연어 이해 및 코드 생성

### Excel 파일의 특징
Excel 파일은 다음과 같은 정형 데이터의 특징을 가집니다:
- **표 형태 구조**: 행과 열로 구성된 정형화된 데이터
- **데이터 타입**: 숫자, 텍스트, 날짜 등 명확한 데이터 타입
- **관계성**: 컬럼 간의 관계와 데이터의 연관성
- **메타데이터**: 컬럼명, 데이터 타입, 제약조건 등 구조적 정보

## Excel 파일 로딩 및 기본 구조 분석
pandas를 활용하여 Excel 파일을 DataFrame으로 로드하고 기본적인 데이터 구조를 파악합니다.

In [ ]:
file_name = "2024학년도+2학기+온라인+수업+중간+강의평가+결과.xlsx"
doc_path = f"{data_path}/{file_name}"

In [ ]:
import pandas as pd

df = pd.read_excel(doc_path)

df.head()

## 데이터 전처리 및 품질 개선

### 컬럼명 전처리
컬럼명에 포함된 개행문자(\n)를 제거합니다.

In [ ]:
# 컬럼명에서 개행문자 제거
df.columns = df.columns.str.replace("\n", " ")
df.head()

## LangChain DataFrame Agent 구축
### 자연어 기반 데이터 분석 시스템 구현

- LangChain의 `create_pandas_dataframe_agent`를 활용하여 AI Agent를 구축합니다.
- LangChain Agent를 생성하여 데이터프레임에 대한 자연어 질의응답을 수행합니다.
- OpenAI GPT-4o-mini 모델을 기반으로 ChatOpenAI 객체를 생성합니다.
- Pandas DataFrame을 다루는 Agent를 생성하고 설정합니다.

In [ ]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

# OpenAI 모델을 사용하는 ChatOpenAI 객체 생성
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

# DataFrame을 다루는 agent 생성
agent = create_pandas_dataframe_agent(
    llm,  # 모델 지정
    df,  # 데이터프레임 지정
    agent_type="tool-calling",
    verbose=True,  # 추론과정 출력
    allow_dangerous_code=True,  # python 코드 실행 허용
)

In [ ]:
agent.run("항공대학 학생들은 '강의 평점'에 대해 어떻게 평가했나요?")

# TXT 파일 전처리 실습
## 다양한 텍스트 파일(.txt) 데이터 추출 및 정제

이 섹션에서는 **다양한 형식의 텍스트 파일(.txt)**로부터 데이터를 추출하고 정제하는 과정을 실습합니다.   

### 실습 목표
- 다양한 형식의 텍스트 파일 처리 방법 학습
- 로그 파일의 구조 분석 및 패턴 추출
- 구분자 기반 텍스트 파일의 파싱 및 구조화
- 텍스트 데이터의 정제 및 전처리 기법 이해

### 실습 예제 파일들
1. **로그 파일 (Apache_2k.log)**
   - Loghub에서 제공하는 Apache web server error log
   - 웹서버 에러 로그의 구조와 패턴 분석
   - 타임스탬프, 로그 레벨, 에러 메시지 등 구조화된 로그 데이터

2. **구분자 기반 파일 (customers.txt)**
   - Sling Academy에서 제공하는 무료 고객 데이터
   - CSV 형태의 구조화된 텍스트 데이터
   - 고객 정보, 구매 내역 등 비즈니스 데이터

### 사용 라이브러리
- **pandas**: CSV 형태 데이터 처리
- **re (정규표현식)**: 로그 파일 패턴 매칭
- **datetime**: 타임스탬프 처리
- **collections**: 데이터 집계 및 분석

### 텍스트 파일의 특징
텍스트 파일은 다음과 같은 다양한 형태로 존재할 수 있습니다:
- **구조화된 텍스트**: CSV, TSV 등 구분자 기반 데이터
- **반구조화된 텍스트**: 로그 파일, 설정 파일 등
- **비구조화된 텍스트**: 일반 문서, 메모 등
- **시계열 데이터**: 타임스탬프가 포함된 로그 데이터

In [ ]:
log_file = f"{data_path}/Apache_2k.log"
with open(log_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

for line in lines[:10]:
    print(line, end="")

## 로그 데이터 전처리 및 구조화
### Apache 웹서버 에러 로그 분석 및 패턴 추출

Loghub에서 제공하는 **Apache web server error log**를 분석하여 웹서버의 운영 상태와 에러 패턴을 파악합니다.

### 로그 파일 정보
- **파일명**: Apache_2k.log
- **내용**: Apache 웹서버의 실제 에러 로그 데이터
- **규모**: 약 52K 줄, 29개의 템플릿 패턴
- **특징**: 원형 그대로의 실제 운영 로그 (타임스탬프, 레벨, PID 등 포함)

### 로그 구조 분석
Apache 로그는 다음과 같은 구조로 구성됩니다:
- **타임스탬프**: [Sun Dec 04 04:47:44 2005] (`[%a %b %d %H:%M:%S %Y]`) 형식의 시간 정보
- **로그 레벨**: `notice`, `error` 등의 로그 중요도 수준
- **프로세스 ID**: `[pid 6725]` 형태의 프로세스 식별자 (선택적)
- **메시지**: 실제 로그 내용 및 에러 설명

### 분석 목표
- **에러 패턴 식별**: 가장 빈번하게 발생하는 에러 유형 파악
- **시간대별 분석**: 서버 부하가 높은 시간대 식별
- **시스템 상태 모니터링**: 웹서버의 전반적인 운영 상태 평가
- **예방적 유지보수**: 반복되는 에러 패턴을 통한 문제 예측

### 처리 방법
1. **정규표현식 파싱**: 로그 라인을 구조화된 데이터로 변환
2. **DataFrame 변환**: pandas를 활용한 효율적인 데이터 분석
3. **시각화**: matplotlib을 통한 직관적인 패턴 분석
4. **통계 분석**: 빈도수, 분포 등 통계적 인사이트 도출

### 정규표현식으로 로그 파싱

In [ ]:
import re

# 정규표현식 패턴 정의
pattern = r"\[(.*?)\] \[(\w+)\] (?:\[pid \d+\] )?(.*)"

parsed_logs = []

for line in lines:
    match = re.match(pattern, line)
    if match:
        timestamp, level, message = match.groups()
        log_dict = {"timestamp": timestamp, "level": level, "message": message.strip()}
        parsed_logs.append(log_dict)

parsed_logs[:5]

### `pandas DataFrame`으로 로그 관리 및 분석

In [ ]:
import pandas as pd

log_df = pd.DataFrame(parsed_logs)
log_df.head()

### 로그 레벨별 분포 시각화

In [ ]:
import matplotlib.pyplot as plt

# 로그 레벨별 카운트 계산
level_counts = log_df["level"].value_counts()

# 막대 그래프 생성
plt.figure(figsize=(6, 4))
plt.bar(level_counts.index, level_counts.values)

# 그래프 꾸미기
plt.title("Log Level Distribution", fontsize=14)
plt.xlabel("Log Level", fontsize=12)
plt.ylabel("Count", fontsize=12)

# 막대 위에 값 표시
for i, v in enumerate(level_counts.values):
    plt.text(i, v, str(v), ha="center", va="bottom")

plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

### 시간대별 로그 발생 건수 분포 및 시각화

In [ ]:
log_df["hour"] = pd.to_datetime(log_df["timestamp"]).dt.hour
print("시간대별 로그 발생 건수:")
print(log_df["hour"].value_counts().sort_index())
print("\n")

In [ ]:
# 시간대별 로그 발생 건수 계산
log_df["hour"] = pd.to_datetime(log_df["timestamp"]).dt.hour
hourly_counts = log_df["hour"].value_counts().sort_index()

# 시각화
plt.figure(figsize=(6, 4))
plt.bar(hourly_counts.index, hourly_counts.values)

# 그래프 꾸미기
plt.title("Hourly Log Distribution", fontsize=14)
plt.xlabel("Hour", fontsize=12)
plt.ylabel("Count", fontsize=12)

plt.xticks(range(24))
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

### 가장 빈번하게 발생하는 메시지 패턴 확인

In [ ]:
print("가장 많이 발생한 메시지 Top 5:")
print(log_df["message"].value_counts().head())

## LangChain의 dataframe agent로 응답 요청

In [ ]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

# OpenAI 모델을 사용하는 ChatOpenAI 객체 생성
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

# DataFrame을 다루는 agent 생성
agent = create_pandas_dataframe_agent(
    llm,  # 모델 지정
    log_df,  # 데이터프레임 지정
    agent_type="tool-calling",
    verbose=True,  # 추론과정 출력
    allow_dangerous_code=True,  # python 코드 실행 허용
)

In [ ]:
agent.run("가장 빈번한 메세지 패턴은 무엇인가요?")

## 구분자 기반 파일 전처리 및 고객 데이터 분석
### CSV 형태 텍스트 파일의 구조화 및 비즈니스 인사이트 도출

Sling Academy에서 제공하는 무료 고객 데이터를 활용하여 구분자 기반 텍스트 파일의 전처리와 비즈니스 분석을 수행합니다.

### 고객 데이터 정보
- **파일명**: customers.txt
- **형태**: 구분자(,) 기반 CSV 형식의 텍스트 파일
- **데이터 크기**: 2,002개의 고객 레코드
- **컬럼 구성**: 13개의 고객 관련 속성

### 데이터 컬럼 구조
고객 데이터는 다음과 같은 정보를 포함합니다:
- **기본 정보**: first_name, last_name, email, phone, address, gender, age
- **등록 정보**: registered (가입일)
- **구매 정보**: orders (주문 건수), spent (총 구매액)
- **개인 정보**: job (직업), hobbies (취미), is_married (결혼 여부)


In [ ]:
import pandas as pd

customer_file = f"{data_path}/customers.txt"
df = pd.read_csv(customer_file, sep=",")

df.head()

### 간단한 Feature Engineering
- 주소에서 도시/주 분리
- 등록일자(registered)를 날짜 객체로 변환
- 총 구매액(spent)을 주문 건수(orders)로 나누어 평균 주문액 계산

#### 컬럼명 정리

In [ ]:
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
df.head()

#### 주소에서 도시/주 분리

In [ ]:
# 주소에서 도시/주 분리하기
df[["city", "state"]] = df["address"].str.extract(r"([^,\n]+),\s*([A-Z]{2})")

# 결과 확인
print("주소 분리 결과:")
print(df[["address", "city", "state"]].head())

#### 등록일자(registered)를 날짜 객체로 변환

In [ ]:
df["registered"] = pd.to_datetime(df["registered"])

# 변환 결과 확인
print("registered 컬럼의 데이터 타입:", df["registered"].dtype)
print("\n등록일자 샘플 데이터:")
print(df["registered"].head())

#### 평균 주문액 컬럼 추가
- 총 구매액(spent)을 주문 건수(orders)로 나누어 평균 주문액 계산

In [ ]:
# 평균 주문액 계산
df["avg_order_amount"] = df["spent"] / df["orders"]

# 결과 확인
print("평균 주문액 계산 결과:")
print(df[["orders", "spent", "avg_order_amount"]].head())

## LangChain의 DataFrame Agent로 응답 요청

In [ ]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

# OpenAI 모델을 사용하는 ChatOpenAI 객체 생성
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

# DataFrame을 다루는 agent 생성
agent = create_pandas_dataframe_agent(
    llm,  # 모델 지정
    df,  # 데이터프레임 지정
    agent_type="tool-calling",
    verbose=True,  # 추론과정 출력
    allow_dangerous_code=True,  # python 코드 실행 허용
    handle_parsing_errors=True,
)

In [ ]:
agent.run("고객의 직업들 중 가장 많은 고객이 종사하는 직업은 무엇인가요?")

# HWP 파일 전처리 실습
## 한글 워드프로세서 문서(.hwp) 데이터 추출 및 정제

이 섹션에서는 **한글 워드프로세서 문서(.hwp)** 파일로부터 데이터를 추출하고 정제하는 과정을 실습합니다.

### 실습 목표
- `olefile` 라이브러리를 활용한 HWP 파일 구조 분석
- HWP 파일의 내부 구조와 스트림 시스템 이해
- 압축된 바이너리 데이터의 해제 및 텍스트 추출
- 한글 문서의 특성을 고려한 텍스트 정제 방법 학습

### 예제 파일 정보
- **파일명**: `[붙임1] 2025년 서울형 R&D 지원사업 통합공고.hwp`
- **내용**: 서울시 R&D 지원사업 관련 공식 공고문
- **특징**: 정부 공고문으로 구조화된 문서, 한글 특화 형식

### 사용 라이브러리
- **olefile**: OLE(Object Linking and Embedding) 구조 파일 읽기
- **zlib**: 압축된 데이터 해제
- **struct**: 바이너리 데이터 구조 처리

### HWP 파일의 특징
HWP 파일은 다음과 같은 복잡한 구조를 가집니다:
- **OLE 구조**: Microsoft의 OLE 표준을 따르는 복합 문서 형식
- **스트림 시스템**: 문서의 각 부분이 독립적인 스트림으로 저장
- **압축 데이터**: 텍스트가 압축된 형태로 저장되어 해제 필요
- **바이너리 형식**: 텍스트가 바이너리 레코드 형태로 저장

## HWP 문서 구조 분석 및 로딩
### OLE 구조를 활용한 HWP 파일 내부 탐색

`olefile` 라이브러리를 활용하여 HWP 파일의 내부 구조를 분석하고, 문서의 각 부분에 접근하는 방법을 학습합니다.

### OLE 구조의 개념
- **OLE(Object Linking and Embedding)**: Microsoft에서 개발한 복합 문서 표준
- **스트림(Stream)**: 파일 시스템의 파일과 유사한 개념으로, 문서의 특정 부분을 담고 있음
- **계층 구조**: 디렉토리처럼 경로로 접근 가능한 구조
- **바이너리 데이터**: 각 스트림은 바이너리 형태로 저장된 데이터

### 주요 스트림 구조
HWP 파일은 다음과 같은 주요 스트림들로 구성됩니다:
- **BodyText/Section0**: 문서의 본문 텍스트 (가장 중요한 부분)
- **DocInfo**: 문서의 메타데이터 정보 (작성자, 생성일 등)
- **FileHeader**: 파일의 기본 정보 및 압축 여부 등
- **PrvText**: 미리보기 텍스트
- **BinData**: 이미지, 표 등 바이너리 데이터
- **Scripts**: 매크로나 스크립트 정보

### 스트림 접근 방법
1. **파일 유효성 검사**: `olefile.isOleFile()` 함수로 HWP 파일 여부 확인
2. **스트림 목록 조회**: `listdir()` 메서드로 모든 스트림 경로 확인
3. **스트림 열기**: `openstream()` 메서드로 특정 스트림에 접근
4. **데이터 읽기**: `read()` 메서드로 바이너리 데이터 추출

### 구조 분석의 중요성
- **데이터 위치 파악**: 원하는 정보가 어느 스트림에 저장되어 있는지 확인
- **압축 정보 확인**: 텍스트가 압축되어 있는지 여부 파악
- **메타데이터 추출**: 문서 정보, 작성자, 생성일 등 부가 정보 수집
- **이미지 처리**: BinData 스트림에서 이미지나 표 데이터 추출

In [ ]:
import olefile

file_name = "[붙임1] 2025년 서울형 R&D 지원사업 통합공고.hwp"
file_path = f"{data_path}/{file_name}"

if olefile.isOleFile(file_path):
    hwp = olefile.OleFileIO(file_path)
    streams = hwp.listdir()
    for stream in streams:
        print(stream)
else:
    print("유효한 HWP 파일이 아닙니다.")

In [ ]:
# 섹션 스트림은 binary로 저장되어 있음
section_path = ["BodyText", "Section0"]
section_data = hwp.openstream(section_path).read()

print(f"Section0 데이터 길이: {len(section_data)} bytes")
print("앞 100바이트 미리보기:", section_data[:100])

## 문서를 문자열로 추출
- olefile 라이브러리를 활용해 HWP의 텍스트를 추출하는 코드는   
[SJKO님의 포스트 '[LLM] Python으로 다양한 문서에서 텍스트 추출하는법'](https://sjkoding.tistory.com/90)을 참고해 작성하였습니다.

In [ ]:
import struct  # 바이너리 데이터 구조 처리를 위한 라이브러리
import zlib  # 압축된 데이터 해제를 위한 라이브러리

import olefile  # OLE 구조의 HWP 파일을 읽기 위한 라이브러리

### 파일 header에서 압축 여부 확인
- HWP 파일 헤더의 36번째 바이트(0-based index): 문서 속성 정보를 포함
- 이 바이트의 첫번째 비트(LSB): 본문의 압축 여부
    - 0(False): 압축되지 않음 / 1(True): 압축됨

In [ ]:
header = hwp.openstream("FileHeader")
header_data = header.read()

# 비트 연산으로 첫 비트 값 확인
is_compressed = (header_data[36] & 1) == 1

# 결과 확인
print(f"문서 압축 여부: {'압축됨' if is_compressed else '압축되지 않음'}")

### 본문(BodyText) 섹션들을 순서대로 처리하기 위한 준비

In [ ]:
# 1. 빈 리스트를 생성하여 섹션 번호를 저장할 공간 확보
nums = []

# 2. 파일의 모든 스트림을 순회하면서 BodyText 섹션 찾기
for stream in streams:
    # 3. 스트림이 BodyText로 시작하는 경우 (본문 섹션인 경우)
    if stream[0] == "BodyText":
        # 4. "Section" 이후의 문자열을 정수로 변환하여 섹션 번호 추출
        # 예: "Section0" -> 0, "Section1" -> 1
        nums.append(int(stream[1][len("Section") :]))

# 5. 추출된 섹션 번호를 정렬하고, 완전한 섹션 경로 문자열 생성
# 예: ["BodyText/Section0", "BodyText/Section1", ...]
sections = ["BodyText/Section" + str(x) for x in sorted(nums)]  # 섹션들을 순서대로 처리하기 위한 경로 목록

### 문서 전체 텍스트 추출 시작

In [ ]:
# 추출된 모든 텍스트를 저장할 빈 문자열 초기화
text = ""

# sections 리스트에 저장된 각 섹션을 순차적으로 처리
for section in sections:
    # 현재 섹션의 스트림을 열어서 바이너리 데이터를 읽음
    bodytext = hwp.openstream(section)
    data = bodytext.read()

    # 파일 헤더에서 확인한 압축 여부에 따라 데이터 처리
    if is_compressed:
        # 압축된 경우 zlib으로 압축 해제
        # -15: raw deflate 압축 방식 중 HWP 파일의 표준 압축 방식
        unpacked_data = zlib.decompress(data, -15)
    else:
        # 압축되지 않은 경우 원본 데이터 그대로 사용
        unpacked_data = data

    # 현재 섹션의 텍스트를 저장할 빈 문자열 초기화
    section_text = ""
    i = 0  # 바이트 단위로 데이터를 읽기 위한 인덱스
    size = len(unpacked_data)  # 처리할 데이터의 전체 크기

    # 바이트 단위로 데이터를 순차 처리
    while i < size:
        # 레코드 헤더를 4바이트 단위로 읽어서 부호 없는 정수로 변환
        # struct.unpack_from으로 바이너리 데이터를 정수로 해석
        header = struct.unpack_from("<I", unpacked_data, i)[0]

        # 비트 연산으로 레코드 타입과 길이 추출
        rec_type = header & 0x3FF  # 하위 10비트를 마스킹하여 레코드 타입 추출
        rec_len = (header >> 20) & 0xFFF  # 상위 12비트를 마스킹하여 레코드 길이 추출

        # 텍스트 레코드(타입 67)인 경우에만 텍스트 추출 수행
        if rec_type in [67]:
            # 헤더(4바이트) 이후부터 레코드 길이만큼의 실제 데이터 추출
            rec_data = unpacked_data[i + 4 : i + 4 + rec_len]
            # UTF-16 인코딩으로 디코딩하여 텍스트 추출
            section_text += rec_data.decode("utf-16")
            # 텍스트 구분을 위한 개행 문자 추가
            section_text += "\n"

        # 현재 레코드의 크기만큼 인덱스 이동 (헤더 4바이트 + 실제 데이터 길이)
        i += 4 + rec_len

    # 현재 섹션의 텍스트를 전체 텍스트에 추가
    text += section_text
    # 섹션 구분을 위한 개행 문자 추가
    text += "\n"

In [ ]:
text

## HWP 텍스트 정제 및 품질 개선
### 바이너리 아티팩트 제거 및 한글 텍스트 최적화

추출된 HWP 텍스트에는 바이너리 데이터 처리 과정에서 발생한 불필요한 문자들과 깨진 데이터가 포함되어 있습니다.    
정규표현식을 활용하여 텍스트를 정제하고 순수한 한글 텍스트만 추출합니다.

### 정제 대상 요소들
1. **제어 문자**: `\x02`, `\x03`, `\x04` 등 HWP 파일 포맷의 내부 구조 바이너리 데이터
2. **깨진 한자**: `捤獥`, `汤捯`, `湰灧` 등 바이너리 데이터가 잘못 디코딩된 문자
3. **확장 ASCII**: 128-255 범위의 비표준 ASCII 문자
4. **특수 제어 문자**: 개행문자(\n)를 제외한 모든 제어 문자

In [ ]:
import re

# 제어 문자 및 확장 ASCII 제거 (\n 제외)
cleaned_text = re.sub(r"[\x00-\x09\x0b-\x1f\x7f-\xff]+", "", text)
# 한글과 기본 문장 부호만 유지
cleaned_text = re.sub(r"[^\x00-\x7F\uAC00-\uD7AF\n]", "", cleaned_text)

print(cleaned_text)

## LangChain Document로 변환

In [ ]:
from langchain.docstore.document import Document

# 텍스트를 LangChain Document 객체로 변환
documents = Document(page_content=cleaned_text, metadata={"title": file_name, "source": file_path})

## olefile로 로드한 문서 RAG test

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LLM component
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Embedding component
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Vectorstore component
embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# 1. Indexing
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 문자 수
    chunk_overlap=200,  # 청크 간 중복되는 문자 수 (문맥 유지를 위함)
    add_start_index=True,  # 원본 문서에서의 시작 위치를 추적할지 여부
)
splits = text_splitter.split_documents([documents])
document_ids = vector_store.add_documents(documents=splits)


def retreive_and_generate(query, vs):
    # 2. Retrieval
    docs = vs.similarity_search(query)
    context_texts = []
    for doc in docs:
        metadata_str = "\n".join(f"{k}: {v}" for k, v in doc.metadata.items())
        context_texts.append(f"Content:\n{doc.page_content}\n\nMetadata:\n{metadata_str}")

    # 3. Generation
    template = """다음 문맥을 바탕으로 사용자의 질문에 자세히 답변해주세요.
    답변은 반드시 한국어로 작성해야 합니다.
    문맥에 관련된 내용이 없다면, '주어진 문맥에서는 해당 질문에 대한 답변을 찾을 수 없습니다.'라고 답변하세요.

    문맥: {context}
    질문: {question}
    응답:"""

    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm

    response = chain.invoke({"context": "\n\n".join(context_texts), "question": query})
    return response.content

In [ ]:
question = "이 지원사업의 신청 자격과 신청 방법을 알려주세요."
print(retreive_and_generate(question, vector_store))

# PDF 파일 전처리 실습
## Portable Document Format(.pdf) 데이터 추출 및 분석

이 섹션에서는 **PDF(Portable Document Format)** 파일로부터 데이터를 추출하고 분석하는 과정을 실습합니다.   
PDF는 기업 문서, 공공 보고서, 논문, 메뉴얼 등 다양한 분야에서 사실상 표준 문서 형식으로 사용되고 있습니다.

### 실습 목표
- 4가지 주요 PDF 처리 라이브러리의 특징과 활용법 비교 학습
- 각 라이브러리별 성능과 한계점 비교 분석

### 예제 파일 정보
- **파일명**: `2024_주요법령해석사례_해설집.pdf`
- **출처**: 법제처
- **내용**: 2024년도 법령해석사례 해설집
- **특징**: 168페이지, 2.4MB, 한글 문서, 법률 전문 문서

### 사용 라이브러리 비교
1. **PyPDF2**: 단순 텍스트 추출, 빠르고 가벼움
2. **PDFMiner.six**: 레이아웃 보존, 구조적 텍스트 추출
3. **PDFPlumber**: 표와 이미지 처리, 고급 레이아웃 분석
4. **Unstructured**: AI 기반 지능형 문서 처리, 복합 요소 처리

### PDF 파일의 특징
PDF 파일은 다음과 같은 복잡한 특성을 가집니다:
- **고정 레이아웃**: 레이아웃을 고정된 형태로 유지하는 장점   
  -> LLM 기반 문서 처리에 복잡한 전처리 요구
- **복합 요소**: 텍스트, 이미지, 표, 링크, 주석 등 다양한 요소 포함
- **일관성 부족**: 문서마다 다른 구조와 형식으로 인한 추출 어려움
- **한글 처리**: 한글 또는 다국어 문서 처리 시 텍스트 깨짐 문제

In [ ]:
file_name = "2024_주요법령해석사례_해설집.pdf"
file_path = f"{data_path}/{file_name}"

# 출력해볼 문자열 길이 지정
MAX_CHARS = 1000

## PyPDF2
- 무료 오픈소스
- 단순 텍스트 추출이 목표일 때 사용 추천
- 설치가 단순하며 가장 빠르고 가벼움
- 표/이미지 처리 불가, 낮은 레이아웃 유지
- LangChain 통합 지원: [관련 링크](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)

In [ ]:
from PyPDF2 import PdfReader  # type: ignore

reader = PdfReader(file_path)
pypdf_text = ""
for page in reader.pages:
    pypdf_text += page.extract_text() or ""
print(pypdf_text[:MAX_CHARS])

## PDFMiner.six
- 무료 오픈소스
- 문자 단위로 정밀한 레이아웃 분석이 필요할 때
- 글자 기반 분석으로 정확도가 높은 편
- 처리 속도는 느린 편
- LangChain 통합 지원: [관련 링크](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PDFMinerLoader.html)

In [ ]:
from pdfminer.high_level import extract_text

pdfminer_text = extract_text(file_path)
print(pdfminer_text[:MAX_CHARS])

## PDFPlumber
- 무료 오픈소스
- 표와 레이아웃이 많은 문서에 효과적임
- 이미지 위치 정보 추출 가능
- 이미지 OCR은 지원하지 않으며 코드가 다소 복잡한 편
- LangChain 통합 지원: [관련 링크](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PDFPlumberLoader.html)

In [ ]:
import pdfplumber

with pdfplumber.open(file_path) as pdf:
    pdfplumber_text = ""
    for page in pdf.pages:
        pdfplumber_text += page.extract_text() or ""
print(pdfplumber_text[:MAX_CHARS])

## Unstructured
- 무료 오픈소스
- 표/이미지 추출에 우수하며 다양한 포맷을 지원
- 설치환경이 복잡하고 모델의 크기가 큼
- LangChain 통합 지원: [관련 링크](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.UnstructuredPDFLoader.html)
- Unstructured library 자체는 무거워서, LangChain에서 PDF Loader로 제공하는 `langchain_community.document_loaders.UnstructuredPDFLoader` 사용

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader(file_path)
unstructured_docs = loader.load()
len(unstructured_docs)

In [ ]:
unstructured_text = unstructured_docs[0].page_content
print(unstructured_text[:MAX_CHARS])

## LangChain Document 객체로 변환
- 4개 라이브러리 중 Unstructured로 추출한 텍스트를 가지고 변환하겠습니다.

In [ ]:
# Unstructured로 추출한 텍스트를 LangChain Document로 변환
from langchain.docstore.document import Document

pdf_document = Document(
    page_content=unstructured_text,
    metadata={"title": file_name, "source": file_path, "extraction_method": "unstructured", "file_type": "pdf"},
)

## RAG 시스템 구축 및 테스트

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LLM component
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Embedding component
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Vectorstore component
embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# 1. Indexing
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 문자 수
    chunk_overlap=200,  # 청크 간 중복되는 문자 수 (문맥 유지를 위함)
    add_start_index=True,  # 원본 문서에서의 시작 위치를 추적할지 여부
)
splits = text_splitter.split_documents([pdf_document])
document_ids = vector_store.add_documents(documents=splits)


def retreive_and_generate(query, vs):
    # 2. Retrieval
    docs = vs.similarity_search(query)
    context_texts = []
    for doc in docs:
        metadata_str = "\n".join(f"{k}: {v}" for k, v in doc.metadata.items())
        context_texts.append(f"Content:\n{doc.page_content}\n\nMetadata:\n{metadata_str}")

    # 3. Generation
    template = """다음 문맥을 바탕으로 사용자의 질문에 자세히 답변해주세요.
    답변은 반드시 한국어로 작성해야 합니다.
    문맥에 관련된 내용이 없다면, '주어진 문맥에서는 해당 질문에 대한 답변을 찾을 수 없습니다.'라고 답변하세요.

    문맥: {context}
    질문: {question}
    응답:"""

    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm

    response = chain.invoke({"context": "\n\n".join(context_texts), "question": query})
    return response.content

In [ ]:
question = "의료법에 대한 해석례는 무엇인가요?"
print(retreive_and_generate(question, vector_store))

##

##